# ELO Scoring Function

In [1]:
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from collections import Counter
import pickle

### Helper Functions

In [2]:
# Given eloA, and eloB, return the probability that A wins
def expected_prob(eloA, eloB):
    return 1 - (1 / (1+10**((eloA-eloB)/400)))

# Given eloA, eloB, constant = k, result of A. Return updated eloA, and eloB values
def update_elo(eloA, eloB, k, res):
    pa = expected_prob(eloA, eloB)
    eloA += (res - pa)*k
    eloB += ((1-res) - (1 - pa))*k
    return eloA, eloB

def percentage(num):
    return round(num,4)*100

### Finding ELO on Datasets

In [126]:
# Example of ELO Calculation

eloA = 1200
eloB = 1000
pa = expected_prob(eloA, eloB)
print('Chance of A to win = ', percentage(pa), '%' )

res = 1 # A wins
eloA, eloB = update_elo(eloA, eloB, 100, res)
print('ELO of A = ', round(eloA, 0))
print('ELO of B = ', round(eloB, 0))

Chance of A to win =  75.97 %
ELO of A =  1224.0
ELO of B =  976.0


For the ELO List, to improve efficiency could add players and update ELO as you run through the matches. For now, it initializes a list of all the players and the code pulls from the list.

In [101]:
def init_elo_df(trn_data):
    elo_df = df({'Player': [], 'Group': [], 'ELO': []})
    start_elo = 1000
    for idx, trn in enumerate(trn_data):
        if idx % 3 == 0:
            for player1 in trn:
                elo_df.loc[len(elo_df.index)] = [player1, 1, start_elo]
            for player2 in trn[player1]:
                elo_df.loc[len(elo_df.index)] = [player2, 2, start_elo]
    return elo_df

# Read pkl data
# trn_data is a list w/ length of 4, and dicts inside w/ length of 12
# 1st level is 4 tournaments, 2nd level is 12 p1's, 3rd level is 12 p2's, 4th level is match histories
file_name = "C:/Users/drago/OneDrive - UW/Ratliff Research/tournaments/smallsoccer-tournaments.pkl"
trn_data = pd.read_pickle(file_name)
elo_df = init_elo_df(trn_data)

# pd.set_option('display.max_colwidth', -1)
# pd.set_option('display.max_rows', None)
# player = player1
# group = 1
# print('Player = ', player)
# print('Group = ', group)
# print('ELO = ', elo_df.loc[(elo_df['Player'] == player) & (elo_df['Group'] == group), 'ELO'].iloc[0])
# print(elo_df)

In [122]:
# Output history dataframe (all data df) + ELO dataframe
hist_df = df({'Tournament': [], 'Player 1': [], 'Player 2': [], 'Result': [], 'New ELO 1': [], 'New ELO 2': []})

k = 100
trnnum = 0
for trn in trn_data:
    trnnum += 1
    for player1 in trn:
        wins = 0
        draws = 0
        games = 0
        eloA = elo_df.loc[(elo_df['Player'] == player1) & (elo_df['Group'] == 1), 'ELO'].iloc[0] # Get ELO A
        for player2 in trn[player1]:
            eloB = elo_df.loc[(elo_df['Player'] == player2) & (elo_df['Group'] == 2), 'ELO'].iloc[0] # Get ELO B
            history = trn[player1][player2]
            for match in history:
                res = 0
                ret1 = match[0]
                ret2 = match[1]
                games += 1
                if (ret1 > ret2):
                    wins += 1
                    res = 1
                elif (ret1 == ret2):
                    draws += 1
                    res = 0.5
                new_eloA, new_eloB = update_elo(eloA, eloB, k, res) 
                elo_df.loc[(elo_df['Player'] == player1) & (elo_df['Group'] == 1), 'ELO'] = new_eloA # Update ELO A
                elo_df.loc[(elo_df['Player'] == player2) & (elo_df['Group'] == 2), 'ELO'] = new_eloB # Update ELO B
                hist_df.loc[len(hist_df.index)] = [trnnum, player1, player2, res, new_eloA, new_eloB]


In [109]:
elo_df.head()

,Player,Group,ELO
0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,975.105770
1,SimplePG-smallsoccer-0_epochs100_batch_size1000_lr0.01_hidden_sizes[64]_-checkpoint.pt,1.0,978.252895
2,"SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[128, 64, 32]_-checkpoint.pt",1.0,1151.866784
3,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[32]_-checkpoint.pt,1.0,963.934473
4,SimplePG-smallsoccer-0_epochs100_batch_size1000_lr0.01_hidden_sizes[32]_-checkpoint.pt,1.0,968.606928


In [119]:
hist_df.head()

,Tournament,Player 1,Player 2,New ELO 1,New ELO 2
0,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,927.683791,1229.537493
1,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1027.683791,1129.537493
2,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1027.683791,1129.537493
3,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1027.683791,1129.537493
4,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1027.683791,1129.537493


In [6]:
# Making ELO List, have a starter df with every player in every group, update ELO in each run,
# have the history pull elo from this list

In [123]:
player = 'SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt'
group = 1
hist_df_players1 = hist_df.groupby('Player 1')
hist_df_player = hist_df_players1.get_group(player)

hist_df_player

,Tournament,Player 1,Player 2,Result,New ELO 1,New ELO 2
0,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,0.0,903.678774,1231.498011
1,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011
2,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011
3,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011
4,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011
5,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,0.0,903.678774,1231.498011
6,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011
7,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,0.0,903.678774,1231.498011
8,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,0.0,903.678774,1231.498011
9,1.0,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,SimplePG-smallsoccer-0_epochs100_batch_size5000_lr0.001_hidden_sizes[64]_-checkpoint.pt,1.0,1003.678774,1131.498011


In [ ]:
stack_pg = raw_df.query("Algo == 'StackPG'")
sns.lineplot(data=stack_pg, x="Epoch", y="agent0_avg_rets")
sns.lineplot(data=stack_pg, x="Epoch", y="agent1_avg_rets").set(title='StackPG Returns', ylabel='Returns')
plt.legend(labels=["Agent 1",  "Agent 2"], loc=0)

figure()

sns.lineplot(data=stack_pg, x="Epoch", y="agent0_loss")
sns.lineplot(data=stack_pg, x="Epoch", y="agent1_loss").set(title='StackPG Loss', ylabel='Loss')
plt.legend(labels=["Agent 1",  "Agent 2"], loc=4)